In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Base folder is one level up from "notebooks"
BASE_DIR = Path("..")

# Excel file path
DATA_FILE = BASE_DIR / "data" / "Data Engineering Use Case Dataset.xlsx"

print(DATA_FILE)


..\data\Data Engineering Use Case Dataset.xlsx


In [2]:
# Read all 3 sheets from Excel
server_meta = pd.read_excel(DATA_FILE, sheet_name="Server_Metadata")
perf_st1 = pd.read_excel(DATA_FILE, sheet_name="Server_Performance_Station1")
perf_st2 = pd.read_excel(DATA_FILE, sheet_name="Server_Performance_Station2")

print("Server_Metadata shape:", server_meta.shape)
print("Station1 shape:", perf_st1.shape)
print("Station2 shape:", perf_st2.shape)

server_meta.head()


Server_Metadata shape: (100, 9)
Station1 shape: (3000, 14)
Station2 shape: (2000, 11)


,Server_ID,Hostname,IP_Address,OS_Type,Server_Location,Admin_Name,Server_Cluster,Admin_Email,Admin_Phone
0,SRV001,host-001,10.0.186.8,Windows,London,George,CL-8,george@xyzcorp.com,+1-202-749-4405
1,SRV002,host-002,10.0.189.206,Linux,Singapore,Alice,CL-2,alice@xyzcorp.com,+1-202-531-6896
2,SRV003,host-003,10.0.219.47,Windows,New York,Ethan,CL-2,ethan@xyzcorp.com,+1-202-853-1213
3,SRV004,host-004,10.0.185.117,Linux,Mumbai,Fiona,CL-6,fiona@xyzcorp.com,+1-202-230-1591
4,SRV005,host-005,10.0.39.146,Windows,New York,Diana,CL-3,diana@xyzcorp.com,+1-202-789-3073


In [3]:
cols_to_drop = ["Config_Version", "Last_Patch_Date", "Deployment_Token"]

perf_st1_clean = perf_st1.drop(columns=cols_to_drop)
perf_st1_clean.head()


,Log_ID,Server_ID,Server_Cluster,Log_Timestamp,CPU_Utilization (%),Memory_Usage (%),Disk_IO (%),Network_Traffic_In (MB/s),Network_Traffic_Out (MB/s),Uptime (Hours),Downtime (Hours)
0,Station1_LOG00001,SRV001,CL-8,2025-08-02 23:30:00,94.63,81.30,44.10,1.67,21.79,506.74,2.77
1,Station1_LOG00002,SRV005,CL-3,2025-08-01 03:12:00,34.98,45.56,58.36,29.42,NaN,157.97,0.23
2,Station1_LOG00003,SRV001,CL-8,2025-08-01 01:33:00,37.41,73.65,74.51,2.79,18.88,606.27,5.85
3,Station1_LOG00004,SRV091,CL-2,2025-08-02 12:56:00,86.86,60.65,23.75,22.67,21.24,893.06,3.84
4,Station1_LOG00005,SRV089,CL-1,2025-08-02 02:50:00,91.59,62.23,50.36,11.64,19.67,198.31,0.22


In [5]:
perf_st1_clean["Source_Station"] = "Station1"
perf_st2["Source_Station"] = "Station2"


In [6]:
perf_st2_clean = perf_st2[perf_st1_clean.columns]

print("Station1 clean:", perf_st1_clean.shape)
print("Station2 clean:", perf_st2_clean.shape)


Station1 clean: (3000, 12)
Station2 clean: (2000, 12)


In [7]:
perf_all = pd.concat([perf_st1_clean, perf_st2_clean], ignore_index=True)
print("Combined performance shape:", perf_all.shape)
perf_all.head()


Combined performance shape: (5000, 12)


,Log_ID,Server_ID,Server_Cluster,Log_Timestamp,CPU_Utilization (%),Memory_Usage (%),Disk_IO (%),Network_Traffic_In (MB/s),Network_Traffic_Out (MB/s),Uptime (Hours),Downtime (Hours),Source_Station
0,Station1_LOG00001,SRV001,CL-8,2025-08-02 23:30:00,94.63,81.30,44.10,1.67,21.79,506.74,2.77,Station1
1,Station1_LOG00002,SRV005,CL-3,2025-08-01 03:12:00,34.98,45.56,58.36,29.42,NaN,157.97,0.23,Station1
2,Station1_LOG00003,SRV001,CL-8,2025-08-01 01:33:00,37.41,73.65,74.51,2.79,18.88,606.27,5.85,Station1
3,Station1_LOG00004,SRV091,CL-2,2025-08-02 12:56:00,86.86,60.65,23.75,22.67,21.24,893.06,3.84,Station1
4,Station1_LOG00005,SRV089,CL-1,2025-08-02 02:50:00,91.59,62.23,50.36,11.64,19.67,198.31,0.22,Station1


In [8]:
perf_all["Log_Timestamp"] = pd.to_datetime(perf_all["Log_Timestamp"])

perf_all["Log_Date"] = perf_all["Log_Timestamp"].dt.date
perf_all["Hour_Of_Day"] = perf_all["Log_Timestamp"].dt.hour

perf_all[["Log_Timestamp", "Log_Date", "Hour_Of_Day"]].head()


,Log_Timestamp,Log_Date,Hour_Of_Day
0,2025-08-02 23:30:00,2025-08-02,23
1,2025-08-01 03:12:00,2025-08-01,3
2,2025-08-01 01:33:00,2025-08-01,1
3,2025-08-02 12:56:00,2025-08-02,12
4,2025-08-02 02:50:00,2025-08-02,2


In [9]:
perf_all.isna().sum()


Log_ID                          0
Server_ID                       0
Server_Cluster                  0
Log_Timestamp                   0
CPU_Utilization (%)             0
Memory_Usage (%)              520
Disk_IO (%)                   271
Network_Traffic_In (MB/s)       0
Network_Traffic_Out (MB/s)    208
Uptime (Hours)                  0
Downtime (Hours)                0
Source_Station                  0
Log_Date                        0
Hour_Of_Day                     0
dtype: int64

In [11]:
numeric_cols = [
    "CPU_Utilization (%)",
    "Memory_Usage (%)",
    "Disk_IO (%)",
    "Network_Traffic_In (MB/s)",
    "Network_Traffic_Out (MB/s)",
    "Uptime (Hours)",
    "Downtime (Hours)",
]


In [12]:
def impute_by_server_then_global(df, group_key, cols):
    df_imputed = df.copy()
    for col in cols:
        # median value per server
        server_median = df_imputed.groupby(group_key)[col].transform("median")
        global_median = df_imputed[col].median()

        # fill with server median
        df_imputed[col] = df_imputed[col].fillna(server_median)
        # if still null, fill with global median
        df_imputed[col] = df_imputed[col].fillna(global_median)
    return df_imputed

perf_all = impute_by_server_then_global(perf_all, "Server_ID", numeric_cols)

perf_all[numeric_cols].isna().sum()


CPU_Utilization (%)           0
Memory_Usage (%)              0
Disk_IO (%)                   0
Network_Traffic_In (MB/s)     0
Network_Traffic_Out (MB/s)    0
Uptime (Hours)                0
Downtime (Hours)              0
dtype: int64

In [13]:
perf_all["Total_Time_Hours"] = perf_all["Uptime (Hours)"] + perf_all["Downtime (Hours)"]

perf_all["Availability (%)"] = np.where(
    perf_all["Total_Time_Hours"] > 0,
    (perf_all["Uptime (Hours)"] / perf_all["Total_Time_Hours"]) * 100,
    np.nan,
)

perf_all[["Uptime (Hours)", "Downtime (Hours)", "Total_Time_Hours", "Availability (%)"]].head()


,Uptime (Hours),Downtime (Hours),Total_Time_Hours,Availability (%)
0,506.74,2.77,509.51,99.456340
1,157.97,0.23,158.20,99.854614
2,606.27,5.85,612.12,99.044305
3,893.06,3.84,896.90,99.571859
4,198.31,0.22,198.53,99.889186


In [14]:
perf_all["Is_High_CPU"] = perf_all["CPU_Utilization (%)"] > 85
perf_all["Is_High_Memory"] = perf_all["Memory_Usage (%)"] > 85
perf_all["Is_High_Disk_IO"] = perf_all["Disk_IO (%)"] > 85

perf_all[["CPU_Utilization (%)", "Is_High_CPU"]].head()


,CPU_Utilization (%),Is_High_CPU
0,94.63,True
1,34.98,False
2,37.41,False
3,86.86,True
4,91.59,True


In [15]:
server_meta["Server_ID"] = server_meta["Server_ID"].astype(str)
perf_all["Server_ID"] = perf_all["Server_ID"].astype(str)


In [16]:
fact_perf = perf_all.merge(
    server_meta,
    on="Server_ID",
    how="left"
)

print("Final table shape:", fact_perf.shape)
fact_perf.head()


Final table shape: (5000, 27)


,Log_ID,Server_ID,Server_Cluster_x,Log_Timestamp,CPU_Utilization (%),Memory_Usage (%),Disk_IO (%),Network_Traffic_In (MB/s),Network_Traffic_Out (MB/s),Uptime (Hours),...,Is_High_Memory,Is_High_Disk_IO,Hostname,IP_Address,OS_Type,Server_Location,Admin_Name,Server_Cluster_y,Admin_Email,Admin_Phone
0,Station1_LOG00001,SRV001,CL-8,2025-08-02 23:30:00,94.63,81.30,44.10,1.67,21.79,506.74,...,False,False,host-001,10.0.186.8,Windows,London,George,CL-8,george@xyzcorp.com,+1-202-749-4405
1,Station1_LOG00002,SRV005,CL-3,2025-08-01 03:12:00,34.98,45.56,58.36,29.42,12.19,157.97,...,False,False,host-005,10.0.39.146,Windows,New York,Diana,CL-3,diana@xyzcorp.com,+1-202-789-3073
2,Station1_LOG00003,SRV001,CL-8,2025-08-01 01:33:00,37.41,73.65,74.51,2.79,18.88,606.27,...,False,False,host-001,10.0.186.8,Windows,London,George,CL-8,george@xyzcorp.com,+1-202-749-4405
3,Station1_LOG00004,SRV091,CL-2,2025-08-02 12:56:00,86.86,60.65,23.75,22.67,21.24,893.06,...,False,False,host-091,10.0.22.9,Windows,London,Hannah,CL-2,hannah@xyzcorp.com,+1-202-151-5538
4,Station1_LOG00005,SRV089,CL-1,2025-08-02 02:50:00,91.59,62.23,50.36,11.64,19.67,198.31,...,False,False,host-089,10.0.225.14,Windows,Frankfurt,Hannah,CL-1,hannah@xyzcorp.com,+1-202-151-5538


In [18]:
fact_perf["Server_Location"] = fact_perf["Server_Location"].fillna("Unknown")
fact_perf["OS_Type"] = fact_perf["OS_Type"].fillna("Unknown")


In [19]:
OUTPUT_DIR = BASE_DIR / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

output_file = OUTPUT_DIR / "curated_server_performance.csv"
fact_perf.to_csv(output_file, index=False)

print("Saved curated file to:", output_file)


Saved curated file to: ..\output\curated_server_performance.csv
